In [2]:
# # data download

import kagglehub

# # Download latest version
covid_path = kagglehub.dataset_download("andyczhao/covidx-cxr2")

In [5]:
import os

os.makedirs("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9/images", exist_ok=True)

for dir_path in os.listdir("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9"):
    if os.path.isdir(os.path.join("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9", dir_path)) and dir_path != "images":
        for file_name in os.listdir(os.path.join("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9", dir_path)):
            src_path = os.path.join("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9", dir_path, file_name)
            dst_path = os.path.join("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9/images", file_name)
            os.rename(src_path, dst_path)

In [6]:
import pandas as pd

train_df = pd.read_csv("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9/train.txt", header=None, names=["Patient ID", "Path", "Label", "Data Source"], sep=" ")
val_df = pd.read_csv("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9/val.txt", header=None, names=["Patient ID", "Path", "Label", "Data Source"], sep=" ")
test_df = pd.read_csv("/users/yliu802/.cache/kagglehub/datasets/andyczhao/covidx-cxr2/versions/9/test.txt", header=None, names=["Patient ID", "Path", "Label", "Data Source"], sep=" ")

for df in [train_df, val_df, test_df]:
    # for Label column, 1 if positive, 0 if negative
    df["Label"] = df["Label"].apply(lambda x: 1 if x == "positive" else 0)


In [ ]:
# combine the train_df, val_df, test_df into one dataframe

full_df = pd.concat([train_df, val_df, test_df], ignore_index=True)
len(full_df)


84818

In [8]:

# randomly split the full_df into train, val, test with stratified sampling
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(full_df, test_size=0.2, stratify=full_df["Label"], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1, stratify=train_val_df["Label"], random_state=42)


train_df.to_csv("../src/covid_train_split.csv", index=False)
val_df.to_csv("../src/covid_val_split.csv", index=False)
test_df.to_csv("../src/covid_test_split.csv", index=False)

In [9]:
import pandas as pd
import numpy as np

def analyze_dataframe(df, path_col="Image Index"):
    print("========== BASIC INFO ==========")
    print("Total samples:", len(df))
    print("Columns:", list(df.columns))
    print()

    if path_col in df.columns:
        print("========== PATH COLUMN CHECK ==========")
        print("Missing paths:", df[path_col].isna().sum())
        print("Unique paths:", df[path_col].nunique())
        print("Example path:", df[path_col].iloc[0])
        print()

    # Identify label columns (everything except path)
    label_cols = [c for c in df.columns if c != path_col]
    print("Label columns:", label_cols)
    print()

    print("========== LABEL SUMMARY ==========")
    print(df[label_cols].describe(include="all"))
    print()

    print("========== NULL VALUES ==========")
    print(df[label_cols].isna().sum())
    print()

    print("========== VALUE COUNTS PER LABEL ==========")
    for col in label_cols:
        print(f"\n--- {col} ---")
        counts = df[col].value_counts(dropna=False)
        print(counts)
        print("Positives (%):", round((df[col] == 1).mean() * 100, 3))
        print("Negatives (%):", round((df[col] == 0).mean() * 100, 3))
        if (-1 in counts.index):
            print("Uncertain (%):", round((df[col] == -1).mean() * 100, 3))
    print()

    print("========== CLASS IMBALANCE RATIO ==========")
    imbalance = (df[label_cols] == 1).sum() / (df[label_cols] == 0).sum()
    print(imbalance)
    print()

    print("========== TOTAL POSITIVES PER LABEL ==========")
    print((df[label_cols] == 1).sum())
    print()

    print("========== TOTAL NEGATIVES PER LABEL ==========")
    print((df[label_cols] == 0).sum())
    print()

    if (df[label_cols] == -1).any().any():
        print("========== TOTAL UNCERTAIN (-1) PER LABEL ==========")
        print((df[label_cols] == -1).sum())
        print()

    print("========== MULTI-LABEL STATS ==========")
    df["num_labels_positive"] = (df[label_cols] == 1).sum(axis=1)
    print(df["num_labels_positive"].describe())
    print("Images with ≥1 positive label:", (df["num_labels_positive"] >= 1).sum())
    print("Images with 0 positive labels:", (df["num_labels_positive"] == 0).sum())
    print()

    print("========== CO-OCCURRENCE: LABELS PER IMAGE ==========")
    print("Histogram of positive label counts:")
    print(df["num_labels_positive"].value_counts().sort_index())
    print()

    print("========== SAMPLE ROWS ==========")
    print(df.head(5))
    print()

    print("========== DONE ==========")
    
analyze_dataframe(train_df, path_col="Path")

========== BASIC INFO ==========
Total samples: 61068
Columns: ['Patient ID', 'Path', 'Label', 'Data Source']

========== PATH COLUMN CHECK ==========
Missing paths: 0
Unique paths: 61068
Example path: A665943-01-13-1901-NA-CHEST_AP_VIEWONLY-82123-2.000000-AP-68954-1-1.jpg

Label columns: ['Patient ID', 'Label', 'Data Source']

========== LABEL SUMMARY ==========
       Patient ID         Label Data Source
count       61068  61068.000000       61068
unique      35580           NaN           8
top       A860070           NaN       bimcv
freq          157           NaN       39341
mean          NaN      0.774366         NaN
std           NaN      0.418002         NaN
min           NaN      0.000000         NaN
25%           NaN      1.000000         NaN
50%           NaN      1.000000         NaN
75%           NaN      1.000000         NaN
max           NaN      1.000000         NaN

========== NULL VALUES ==========
Patient ID     0
Label          0
Data Source    0
dtype: int64

======

In [10]:
analyze_dataframe(val_df, path_col="Path")

========== BASIC INFO ==========
Total samples: 6786
Columns: ['Patient ID', 'Path', 'Label', 'Data Source']

========== PATH COLUMN CHECK ==========
Missing paths: 0
Unique paths: 6786
Example path: sub-S315812_ses-E33352_run-1_bp-chest_vp-pa_dx-corrected.png

Label columns: ['Patient ID', 'Label', 'Data Source']

========== LABEL SUMMARY ==========
       Patient ID        Label Data Source
count        6786  6786.000000        6786
unique       5614          NaN           8
top       A860070          NaN       bimcv
freq           25          NaN        4371
mean          NaN     0.774388         NaN
std           NaN     0.418015         NaN
min           NaN     0.000000         NaN
25%           NaN     1.000000         NaN
50%           NaN     1.000000         NaN
75%           NaN     1.000000         NaN
max           NaN     1.000000         NaN

========== NULL VALUES ==========
Patient ID     0
Label          0
Data Source    0
dtype: int64

========== VALUE COUNTS PER LAB

In [11]:
analyze_dataframe(test_df, path_col="Path")

========== BASIC INFO ==========
Total samples: 16964
Columns: ['Patient ID', 'Path', 'Label', 'Data Source']

========== PATH COLUMN CHECK ==========
Missing paths: 0
Unique paths: 16964
Example path: sub-S07629_ses-E13499_run-1_bp-chest_vp-pa_dx-corrected.png

Label columns: ['Patient ID', 'Label', 'Data Source']

========== LABEL SUMMARY ==========
       Patient ID         Label Data Source
count       16964  16964.000000       16964
unique      12515           NaN           8
top       A860070           NaN       bimcv
freq           54           NaN       10920
mean          NaN      0.774405         NaN
std           NaN      0.417986         NaN
min           NaN      0.000000         NaN
25%           NaN      1.000000         NaN
50%           NaN      1.000000         NaN
75%           NaN      1.000000         NaN
max           NaN      1.000000         NaN

========== NULL VALUES ==========
Patient ID     0
Label          0
Data Source    0
dtype: int64

========== VALUE C